In [14]:
import pandas as pd
import os
#from .ZBMathJournal import ZBMathJournal
from ZBMathJournal import ZBMathJournal
from ZBMathAuthor import ZBMathAuthor
from misc import get_info_from_doi
from wikibaseintegrator.wbi_enums import ActionIfExists
from mardiclient import MardiClient
import traceback
import time

In [2]:
df = pd.read_csv('differences_zbmath_data_dump20231221_TO_zbmath_data_dump20240912.csv', sep='\t')

/tmp/ipykernel_18828/3030566250.py:1: DtypeWarning: Columns (13,16,17,22,25,29,31,32) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('differences_zbmath_data_dump20231221_TO_zbmath_data_dump20240912.csv', sep='\t')


In [3]:
#important: if a value in the new column is NaN, that should mean that the value didn't change

In [4]:
list(df)

['de_number',
 'creation_date',
 'author',
 'author_ids',
 'document_title',
 'source',
 'classifications',
 'language',
 'links',
 'keywords',
 'doi',
 'publication_year',
 'serial',
 'zbl_id',
 'review_text',
 'review_sign',
 'reviewer_id',
 'creation_date_new',
 'author_new',
 'author_ids_new',
 'document_title_new',
 'source_new',
 'classifications_new',
 'language_new',
 'links_new',
 'keywords_new',
 'doi_new',
 'publication_year_new',
 'serial_new',
 'zbl_id_new',
 'review_text_new',
 'review_sign_new',
 'reviewer_id_new']

In [5]:
df.iloc[2].author

'zbMATH Open Web Interface contents unavailable due to conflicting licenses.'

In [6]:
df[(df.review_text != df.review_text_new)][['de_number','review_text', 'review_text_new']]

,de_number,review_text,review_text_new
0,1566037,"Let \(G\) be a topological group, \(M\) be a u...",NaN
1,1566038,This paper deals with manifolds in the categor...,This paper deals with manifolds in the categor...
2,1566074,zbMATH Open Web Interface contents unavailable...,NaN
3,1566075,zbMATH Open Web Interface contents unavailable...,NaN
4,1566076,zbMATH Open Web Interface contents unavailable...,NaN
...,...,...,...
4519839,7680989,zbMATH Open Web Interface contents unavailable...,NaN
4519840,46730,zbMATH Open Web Interface contents unavailable...,Unaltered reprint of the first ed. (1962). See...
4519841,3302617,zbMATH Open Web Interface contents unavailable...,NaN
4519842,7569649,Associated to the quantum multiplication in th...,NaN


In [7]:
df[(df.review_sign != df.review_sign_new)][['de_number','review_sign', 'review_sign_new']]

,de_number,review_sign,review_sign_new
0,1566037,Vladimir L. Popov (Moskva),Vladimir L. Popov
1,1566038,Jouni Luukkainen (Helsinki),Jouni Luukkainen
2,1566074,NaN,NaN
3,1566075,NaN,NaN
4,1566076,NaN,NaN
...,...,...,...
4519839,7680989,NaN,NaN
4519840,46730,NaN,NaN
4519841,3302617,E. Goldhagen,NaN
4519842,7569649,Richárd Rimányi (Chapel Hill),Richárd Rimányi


In [8]:
df[(df.reviewer_id != df.reviewer_id_new)][['de_number','reviewer_id', 'reviewer_id_new']]

,de_number,reviewer_id,reviewer_id_new
0,1566037,popov.vladimir-leonidovich,NaN
1,1566038,luukkainen.jouni,NaN
2,1566074,NaN,NaN
3,1566075,NaN,NaN
4,1566076,NaN,NaN
...,...,...,...
4519839,7680989,NaN,NaN
4519840,46730,NaN,NaN
4519841,3302617,NaN,NaN
4519842,7569649,rimanyi.richard,NaN


In [9]:
df[(df.publication_year != df.publication_year_new)][['de_number','publication_year', 'publication_year_new', 'creation_date']]

,de_number,publication_year,publication_year_new,creation_date
0,1566037,2001,NaN,2002-01-02T23:00:00Z
1,1566038,2001,NaN,2001-07-23T22:00:00Z
2,1566074,1998,NaN,2001-02-19T23:00:00Z
3,1566075,1998,NaN,2001-02-19T23:00:00Z
4,1566076,1999,NaN,2001-02-19T23:00:00Z
...,...,...,...,...
4519839,7680989,2003,NaN,2023-04-28T17:12:52Z
4519840,46730,1990,NaN,1992-09-17T22:00:00Z
4519841,3302617,1969,NaN,0001-01-01T00:00:00
4519842,7569649,2022,NaN,2022-08-09T11:56:27Z


In [21]:
username = os.environ.get('MARDI_USERNAME')
password = os.environ.get('MARDI_PASSWORD')
mc = MardiClient(user=username, password=password)
error_string = 'zbMATH Open Web Interface contents unavailable due to conflicting licenses.'

label_id_dict = {}
label_id_dict["de_number_prop"] = mc.get_local_id_by_label( #de_number_prop
    "zbMATH DE Number", "property"
)
label_id_dict["keyword_prop"] = mc.get_local_id_by_label( #keyword_prop
    "zbMATH Keywords", "property"
)
label_id_dict["review_prop"] = mc.get_local_id_by_label("review text", "property")
label_id_dict["mardi_profile_type_prop"] = mc.get_local_id_by_label("MaRDI profile type", "property")
label_id_dict["mardi_publication_profile_item"] = mc.get_local_id_by_label(
    "Publication", "item"
)[0]
label_id_dict["mardi_person_profile_item"] = mc.get_local_id_by_label("Person", "item")[0]

property_dict = {"document_title":"P159","classifications":"P226","keywords":"P1450","publication_year":"P28","serial":"wdt:P200","links":"P205",
     "doi":"P27","authors":"P16","reviewers":"P1447","review_text":"P1448", "arxiv_id":"P21"}

def is_empty(val):
    if isinstance(val,str):
        val = val.strip()
    if pd.isna(val):
        return True
    elif not val:
        return True
    elif val.strip() == error_string:
        return True
    elif val == "0001-01-01T00:00:00Z":
        return True
    elif val == "None":
        return True
    return False

def generate_authors(author_strings,author_ids):
    authors = []
    for a, a_id in zip(author_strings, author_ids):
        if is_empty(a_id):
            continue
        if is_empty(a):
            a = ""
        for attempt in range(5):
            try:
                author = ZBMathAuthor(
                    integrator=mc,
                    name=a,
                    zbmath_author_id=a_id,
                    label_id_dict=label_id_dict,
                )
                if author.exists():
                    print(f"Author {a} exists!")
                    author_id = author.QID
                    #potentially, the name is wrong, then it needs to be fixed
                    author_item = mc.item.get(entity_id=author_id)
                    label = author_item.labels.values['en'].value
                    if label != a:
                        author_item.labels.set(language="en", value=a)
                        author_item.write()
                else:
                    print(f"Creating author {a}")
                    author_id = author.create()
            except Exception as e:
                print(f"Exception: {e}, sleeping")
                print(traceback.format_exc())
                time.sleep(120)
            else:
                break
        else:
            sys.exit("Uploading author did not work after retries!")
        authors.append(author_id)
    return authors

#author_ids will be used together with authors
old_cols = [x for x in list(df) if not x.endswith("_new") and x not in ["de_number","source", "zbl_id", "language"]]
for _, row in df.iterrows():
    if row["de_number"] in [1566037]:
        continue
    authors_done = False
    reviewers_done=False
    change_dict = {}
    for col in old_cols:
        print(col)
        old_val = row[col]
        new_val = row[col+"_new"]
        if not is_empty(new_val) and old_val!=new_val:
            # if col == "creation_date":
            #     change_dict[col] = f"{new_val.split('T')[0]}T00:00:00Z"
            if col == "document_title":
                change_dict[col] = new_val.strip()
            elif col == "classifications":
                change_dict[col] = new_val.strip().split(";")
            # elif col == "language":
            #     change_dict[col] = new_val.strip()
            elif col == "keywords":
                change_dict[col] = [x.strip() for x in new_val.split(";")]
            elif col == "publication_year":
                change_dict[col] = f"+{new_val.strip()}-00-00T00:00:00Z"
            elif col == "serial":
                #I don't need to check the doi here because this means
                #that there definitely is a serial
                new_val = new_val.split(";")[-1].strip()
                if new_val == old_val.split(";")[-1].strip():
                    continue
                for attempt in range(5):
                            try:
                                journal_item = ZBMathJournal(
                                    integrator=mc, name=new_val
                                )
                                if journal_item.exists():
                                    print(f"Journal {journal_string} exists!")
                                    journal = journal_item.QID
                                else:
                                    print(f"Creating journal {new_val}")
                                    journal = journal_item.create()
                            except Exception as e:
                                print(f"Exception: {e}, sleeping")
                                print(traceback.format_exc())
                                time.sleep(120)
                            else:
                                break
                else:
                    sys.exit("Uploading journal did not work after retries!")
                change_dict[col] = journal
            elif col == "links":
                pattern = re.compile(
                        r"^([a-z][a-z\d+.-]*):([^][<>\"\x00-\x20\x7F])+$"
                    )
                links = new_val.split(";")
                links = [
                    x.strip() for x in links if (pattern.match(x) and "http" in x)
                ]
                arxiv_prefix = "https://arxiv.org/abs/"
                arxiv_id = None
                for l in links:
                    if arxiv_prefix in l:
                        arxiv_id = l.removeprefix(arxiv_prefix)
                        links.remove(l)
                        change_dict["arxiv_id"] = arxiv_id
                        break
                change_dict[col] = links
            elif col == "doi":
                new_val = new_val.strip()
                if is_empty(row["document_title"]) and is_empty(row["document_title_new"]):
                    document_title = get_info_from_doi(
                            doi=new_val, key="document_title"
                        )
                    if document_title:
                        change_dict["document_title"] = document_title
                if is_empty(row["serial"]) and is_empty(row["serial_new"]):
                    journal_string = get_info_from_doi(
                            doi=new_val, key="journal"
                        )
                    if journal_string:
                        for attempt in range(5):
                            try:
                                journal_item = ZBMathJournal(
                                    integrator=mc, name=journal_string
                                )
                                if journal_item.exists():
                                    print(f"Journal {journal_string} exists!")
                                    journal = journal_item.QID
                                else:
                                    print(f"Creating journal {journal_string}")
                                    journal = journal_item.create()
                            except Exception as e:
                                print(f"Exception: {e}, sleeping")
                                print(traceback.format_exc())
                                time.sleep(120)
                            else:
                                break
                        else:
                            sys.exit("Uploading journal did not work after retries!")
                        change_dict["serial"] = journal
                change_dict[col] = new_val
            elif col == "authors":
                if authors_done:
                    continue
                author_strings = [x.strip() for x in  new_val.split(";")]
                if not is_empty(row["author_ids_new"]):
                    author_ids = [x.strip() for x in row["author_ids_new"].split(";")]
                elif not is_empty(row["author_ids"]):
                    author_ids = [x.strip() for x in row["author_ids"].split(";")]
                else:
                    continue
                authors = generate_authors(author_strings=author_strings,author_ids=author_ids)
                if authors: change_dict["authors"] = authors   
                authors_done = True
            elif col == "author_ids":
                if authors_done:
                    continue
                author_ids = [x.strip() for x in new_val.split(";")]
                if not is_empty(row["authors_new"]):
                    author_strings = [x.strip() for x in row["authors_new"].split(";")]
                elif not is_empty(row["authors"]):
                    author_strings = [x.strip() for x in row["authors"].split(";")]
                else:
                    author_strings = [""] * len(author_ids)
                authors = generate_authors(author_strings=author_strings,author_ids=author_ids)
                if authors: change_dict["authors"] = authors
                authors_done = True
            elif col == "review_sign":
                if reviewers_done:
                    continue
                reviewer_strings = [x.strip() for x in  new_val.split(";")]
                if not is_empty(row["reviewer_id_new"]):
                    reviewer_ids = [x.strip() for x in row["reviewer_id_new"].split(";")]
                elif not is_empty(row["reviewer_id"]):
                    reviewer_ids = [x.strip() for x in row["reviewer_id"].split(";")]
                else:
                    continue
                reviewers = generate_authors(author_strings=reviewer_strings,author_ids=reviewer_ids)
                if reviewers: change_dict["reviewers"] = reviewers
                reviewers_done = True
            elif col == "reviewer_id":
                if reviewers_done:
                    continue
                reviewer_ids = [x.strip() for x in new_val.split(";")]
                if not is_empty(row["review_sign_new"]):
                    reviewer_strings = [x.strip() for x in row["review_sign_new"].split(";")]
                elif not is_empty(row["review_sign"]):
                    reviewer_strings = [x.strip() for x in row["review_sign"].split(";")]
                else:
                    reviewer_strings = [""] * len(reviewer_ids)
                reviewers = generate_authors(author_strings=reviewer_strings,author_ids=reviewer_ids)
                if reviewers: change_dict["reviewers"] = reviewers
                reviewers_done = True
            elif col == "review_text":
                change_dict[col] = new_val.strip()
    print(change_dict)
    if change_dict:
        de_number = row["de_number"]
        qid = mc.search_entity_by_value("P1451", str(de_number))[0]
        if not qid:
            print(f"No item for de number {de_number}")
            print(wtf)
        item = mc.item.get(qid)
        arxiv_switch = False
        claim_list = []
        for key, val in change_dict.items():
            if key == "document_title":
                item.labels.set(language="en", value=val)
            elif key == "arxiv_id":
                arxiv_switch = True
                continue
            prop = property_dict[key]
            if isinstance(val, list):
                for v in val:
                    claim = mc.get_claim(prop, v)
                    claim_list.append(claim)
            else:
                claim = mc.get_claim(prop, val)
                claim_list.append(claim)
            item.add_claims(claim_list,ActionIfExists.REPLACE_ALL)
        item.write()
        if arxiv_switch:
            arxiv_paper = search_item_by_property(property_id = property_dict["arxiv_id"], value=change_dict["arxiv_id"])
            if arxiv_paper:
                arxiv_item = mc.item.get(arxiv_paper)
                claim = mc.claim.get(property_dict["arxiv_id"],qid)
                arxiv_item.write()
                
            #find paper by arxiv id
            #add preprint_of property to it
            
        print(de_number)
        print(change_dict)
        print(a)

creation_date
author
author_ids
document_title
classifications
links
keywords
doi
publication_year
serial
review_text
review_sign
Author found with QID Q161374
Author Jouni Luukkainen exists!


Service unavailable (HTTP Code 502). Sleeping for 60 seconds.
Service unavailable (HTTP Code 502). Sleeping for 60 seconds.


reviewer_id
{'review_text': "This paper deals with manifolds in the categories LIP of locally bi-Lipschitz homeomorphisms and LQC of locally quasiconformal (or equivalently locally quasisymmetric) homeomorphisms. The following theorem consists of representative special cases of the main results of the paper. Let CAT be LIP or LQC, let \\(0\\leq q\\leq n\\) with \\(n,q\\neq 4\\), let \\(M\\) be a topological \\(n\\)-manifold without boundary, and let \\(Q\\) be a closed (as a subset) locally flat \\(q\\)-submanifold of \\(M\\) without boundary. Then the pair \\((M,Q)\\) can be given an essentially unique locally CAT flat CAT manifold pair structure as follows.NEWLINENEWLINENEWLINE(1) There is a CAT structure on \\(M\\) for which \\(Q\\) is a locally CAT flat submanifold of \\(M\\).NEWLINENEWLINENEWLINE(2) If \\({\\mathcal A}\\) and \\({\\mathcal B}\\) are two CAT structures on \\(M\\) satisfying (1), then there is an arbitrarily small isotopy of \\((M,Q)\\) from the identity to a CAT ho

NameError: name 'a' is not defined

In [19]:
mc.get_claim("wdt:P1447", 'Q253168')

<Item @5af00 _Claim__mainsnak=<Snak @5b140 _Snak__snaktype=<WikibaseSnakType.KNOWN_VALUE: 'value'> _Snak__property_number='P1447' _Snak__hash=None _Snak__datavalue={'value': {'entity-type': 'item', 'numeric-id': 253168, 'id': 'Q253168'}, 'type': 'wikibase-entityid'} _Snak__datatype='wikibase-item'> _Claim__type='statement' _Claim__qualifiers=<Qualifiers @59070 _Qualifiers__qualifiers={}> _Claim__qualifiers_order=[] _Claim__id=None _Claim__rank=<WikibaseRank.NORMAL: 'normal'> _Claim__removed=False _Claim__references=<References @587d0 _References__references=[]>>

In [10]:
change_dict

NameError: name 'change_dict' is not defined

In [10]:
df[df.de_number == ""]

,de_number,creation_date,author,author_ids,document_title,source,classifications,language,links,keywords,...,language_new,links_new,keywords_new,doi_new,publication_year_new,serial_new,zbl_id_new,review_text_new,review_sign_new,reviewer_id_new
0,1566037,2002-01-02T23:00:00Z,"Önder, Turgut",onder.turgut,Equivariant cross sections of complex Stiefel ...,"Topology Appl. 109, No. 1, 107-125 (2001).",57R91;55R91;55R50;57S25;57R25;55R25;22A25,English,NaN,complex Stiefel manifolds;equivariant fibre sp...,...,NaN,NaN,NaN,NaN,NaN,Topology and its Applications,NaN,NaN,Vladimir L. Popov,NaN
1,1566038,2001-07-23T22:00:00Z,"Luukkainen, Jouni",luukkainen.jouni,Lipschitz and quasiconformal approximation of ...,"Topology Appl. 109, No. 1, 1-40 (2001).",57N35;30C65;57R12,English,NaN,quasisymmetric;locally flat;diffeomorphism,...,NaN,NaN,NaN,NaN,NaN,Topology and its Applications,NaN,This paper deals with manifolds in the categor...,Jouni Luukkainen,NaN
2,1566074,2001-02-19T23:00:00Z,zbMATH Open Web Interface contents unavailable...,alexandru.victor;gosoniu.marin,zbMATH Open Web Interface contents unavailable...,zbMATH Open Web Interface contents unavailable...,11-01,zbMATH Open Web Interface contents unavailable...,NaN,NaN,...,Romanian,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1566075,2001-02-19T23:00:00Z,zbMATH Open Web Interface contents unavailable...,arsinte.vasile,zbMATH Open Web Interface contents unavailable...,zbMATH Open Web Interface contents unavailable...,26-01,zbMATH Open Web Interface contents unavailable...,NaN,NaN,...,Romanian,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1566076,2001-02-19T23:00:00Z,zbMATH Open Web Interface contents unavailable...,badescu.lucian,zbMATH Open Web Interface contents unavailable...,zbMATH Open Web Interface contents unavailable...,51-01;53-01,zbMATH Open Web Interface contents unavailable...,NaN,NaN,...,Romanian,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4519839,7680989,2023-04-28T17:12:52Z,zbMATH Open Web Interface contents unavailable...,deutsch.david,zbMATH Open Web Interface contents unavailable...,zbMATH Open Web Interface contents unavailable...,81-02;81P05;00A30,zbMATH Open Web Interface contents unavailable...,NaN,NaN,...,French,NaN,NaN,NaN,NaN,NaN,1509.81002,NaN,NaN,NaN
4519840,46730,1992-09-17T22:00:00Z,zbMATH Open Web Interface contents unavailable...,rudin.walter,zbMATH Open Web Interface contents unavailable...,zbMATH Open Web Interface contents unavailable...,43-02;43A75;43A25;43A22,zbMATH Open Web Interface contents unavailable...,NaN,harmonic analysis on locally compact Abelian g...,...,English,NaN,NaN,NaN,NaN,NaN,698.43001,Unaltered reprint of the first ed. (1962). See...,NaN,NaN
4519841,3302617,0001-01-01T00:00:00,zbMATH Open Web Interface contents unavailable...,lions.jacques-louis,zbMATH Open Web Interface contents unavailable...,zbMATH Open Web Interface contents unavailable...,NaN,zbMATH Open Web Interface contents unavailable...,NaN,partial differential equations,...,French,NaN,partial differential equations;nonlinear bound...,NaN,NaN,NaN,189.40603,NaN,NaN,NaN
4519842,7569649,2022-08-09T11:56:27Z,"Okounkov, A.; Smirnov, A.",okounkov.andrei-yuryevich;smirnov.andrey-v,Quantum difference equation for Nakajima varie...,"Invent. Math. 229, No. 3, 1203-1299 (2022).",14N35;14C35;19L47;17B37;14C05;17B67,English,https://arxiv.org/abs/1602.09007,\(K\)-theory of Nakajima quiver varieties;quas...,...,NaN,NaN,NaN,NaN,NaN,Inventiones Mathematicae,1504.14096,NaN,Richárd Rimányi,NaN


In [ ]:
print(mc.get_local_id_by_label(
    "Publication", "item"
)[0])

In [ ]:
mc.search_entity_by_value(
                "wdt:P1556", "popov.vladimir-leonidovich"
            )

In [4]:
from mardiclient import MardiClient
import os 

zbmath_id = "1566037"
user = os.environ['MARDI_USER']
password = os.environ['MARDI_PW']
mc = MardiClient(user=user, password=password)
#read user and pw from env vars
qid = mc.search_entity_by_value("P1451", zbmath_id)[0]
my_item = mc.item.get(entity_id=qid)

In [14]:
list(df)

['de_number',
 'creation_date',
 'author',
 'author_ids',
 'document_title',
 'source',
 'classifications',
 'language',
 'links',
 'keywords',
 'doi',
 'publication_year',
 'serial',
 'zbl_id',
 'review_text',
 'review_sign',
 'reviewer_id',
 'creation_date_new',
 'author_new',
 'author_ids_new',
 'document_title_new',
 'source_new',
 'classifications_new',
 'language_new',
 'links_new',
 'keywords_new',
 'doi_new',
 'publication_year_new',
 'serial_new',
 'zbl_id_new',
 'review_text_new',
 'review_sign_new',
 'reviewer_id_new']

In [4]:
compare_large_dfs(
        data_file_1="../../../../zbmath_data_dump20231221-160126.csv",
        data_file_2='../../../../zbmath_data_dump20240912-142641.csv',
        hash_file_1='../../../../zbmath_data_dump20231221-160126_hash.csv',
        hash_file_2="../../../../zbmath_data_dump20240912-142641_hash.csv",
        de_col="de_number",
        chunksize=50_000,
        output_file="differences.csv"
    )

Reading hash files into memory...
Found 4519844 de_numbers with different hashes.


/tmp/ipykernel_29963/3859913072.py:75: DtypeWarning: Columns (17) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk2 in df2_iter:
/tmp/ipykernel_29963/3859913072.py:75: DtypeWarning: Columns (7,8,9,12,14,15,16,17) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk2 in df2_iter:
/tmp/ipykernel_29963/3859913072.py:75: DtypeWarning: Columns (17) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk2 in df2_iter:
/tmp/ipykernel_29963/3859913072.py:75: DtypeWarning: Columns (7,8,9,12,14,15,16,17) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk2 in df2_iter:
/tmp/ipykernel_29963/3859913072.py:75: DtypeWarning: Columns (17) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk2 in df2_iter:
/tmp/ipykernel_29963/3859913072.py:75: DtypeWarning: Columns (7,8,9,12,14,15,16,17) have mixed types. Specify dtype option on import o

Done! Differences written to 'differences.csv'.


In [9]:
old_hash = pd.read_csv('../../../../zbmath_data_dump20231221-160126_hash.csv', sep='\t')
new_hash = pd.read_csv('../../../../zbmath_data_dump20240912-142641_hash.csv', sep='\t')

In [12]:
old_hash.columns

Index(['de_number', 'hash'], dtype='object')

In [5]:
old_ones_not_in_new = old_de - new_de 
new_ones_not_in_old = new_de - old_de

In [5]:
test = pd.read_csv("differences.csv", sep="\t", nrows=200)

In [6]:
pd.set_option('display.max_columns', None)

In [7]:
test

,creation_date,author,author_ids,document_title,source,classifications,language,links,keywords,doi,publication_year,serial,zbl_id,review_text,review_sign,reviewer_id,creation_date_new,author_new,author_ids_new,document_title_new,source_new,classifications_new,language_new,links_new,keywords_new,doi_new,publication_year_new,serial_new,zbl_id_new,review_text_new,review_sign_new,reviewer_id_new
0,2002-01-02T23:00:00Z,"Önder, Turgut",onder.turgut,Equivariant cross sections of complex Stiefel ...,"Topology Appl. 109, No. 1, 107-125 (2001).",57R91;55R91;55R50;57S25;57R25;55R25;22A25,English,NaN,complex Stiefel manifolds;equivariant fibre sp...,10.1016/S0166-8641(99)00149-2,2001,"Elsevier (North-Holland), Amsterdam;Topology a...",976.57032,"Let \(G\) be a topological group, \(M\) be a u...",Vladimir L. Popov (Moskva),popov.vladimir-leonidovich,0001-01-01T00:00:00Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Topology and its Applications,NaN,NaN,Vladimir L. Popov,NaN
1,2001-07-23T22:00:00Z,"Luukkainen, Jouni",luukkainen.jouni,Lipschitz and quasiconformal approximation of ...,"Topology Appl. 109, No. 1, 1-40 (2001).",57N35;30C65;57R12,English,NaN,quasisymmetric;locally flat;diffeomorphism,10.1016/S0166-8641(99)00145-5,2001,"Elsevier (North-Holland), Amsterdam;Topology a...",964.57023,This paper deals with manifolds in the categor...,Jouni Luukkainen (Helsinki),luukkainen.jouni,0001-01-01T00:00:00Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Topology and its Applications,NaN,This paper deals with manifolds in the categor...,Jouni Luukkainen,NaN
2,2001-02-19T23:00:00Z,zbMATH Open Web Interface contents unavailable...,alexandru.victor;gosoniu.marin,zbMATH Open Web Interface contents unavailable...,zbMATH Open Web Interface contents unavailable...,11-01,zbMATH Open Web Interface contents unavailable...,NaN,NaN,NaN,1998,zbMATH Open Web Interface contents unavailable...,1079.11500,zbMATH Open Web Interface contents unavailable...,NaN,NaN,0001-01-01T00:00:00Z,"Alexandru, Victor;Goşoniu, Marin",NaN,Elements of number theory,Bucuresti: Editura Universităţii Bucureşti (IS...,NaN,Romanian,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2001-02-19T23:00:00Z,zbMATH Open Web Interface contents unavailable...,arsinte.vasile,zbMATH Open Web Interface contents unavailable...,zbMATH Open Web Interface contents unavailable...,26-01,zbMATH Open Web Interface contents unavailable...,NaN,NaN,NaN,1998,zbMATH Open Web Interface contents unavailable...,1079.26500,zbMATH Open Web Interface contents unavailable...,NaN,NaN,0001-01-01T00:00:00Z,"Arsinte, Vasile",NaN,Elementary problems of integral calculus,Bucureşti: Editura Universităţii Bucureşti (IS...,NaN,Romanian,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2001-02-19T23:00:00Z,zbMATH Open Web Interface contents unavailable...,badescu.lucian,zbMATH Open Web Interface contents unavailable...,zbMATH Open Web Interface contents unavailable...,51-01;53-01,zbMATH Open Web Interface contents unavailable...,NaN,NaN,NaN,1999,zbMATH Open Web Interface contents unavailable...,1130.51300,zbMATH Open Web Interface contents unavailable...,NaN,NaN,0001-01-01T00:00:00Z,"Bădescu, Lucian",NaN,Lectures on geometry,Bucureşti: Editura Universităţii din Bucureşti...,NaN,Romanian,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,2001-02-19T23:00:00Z,zbMATH Open Web Interface contents unavailable...,newborn.monty,zbMATH Open Web Interface contents unavailable...,zbMATH Open Web Interface contents unavailable...,68T15;68-01,zbMATH Open Web Interface contents unavailable...,NaN,predicate calculus;inference procedures;semant...,NaN,2001,zbMATH Open Web Interface contents unavailable...,963.68176,zbMATH Open Web Interface contents unavailable...,Doina Tătar (Cluj),tatar.doina,0001-01-01T00:00:00Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Doina Tătar,NaN
196,2001-02-19T23:00:00Z,zbMATH Open Web Interface contents unavailable...,fru

In [8]:
test.shape

(200, 32)

In [7]:
len(new_ones_not_in_old)

319375

In [8]:
import csv
import pandas as pd

def read_de_numbers_csv(filename, de_column_name="de_number"):
    de_values = set()
    with open(filename, "r", encoding="utf-8") as f:
        reader = csv.DictReader(f, delimiter="\t")
        for row in reader:
            de_values.add(row[de_column_name])
    return de_values

def subset_large_file_by_ids(
    input_file, 
    output_file, 
    de_values, 
    id_column="de_number", 
    chunksize=100000
):
    """
    Reads input_file in chunks, filters rows where id_column is in de_values,
    and writes them to output_file without loading the entire file into memory.
    """
    # You can tune chunksize depending on your memory constraints.
    first_chunk = True
    with pd.read_csv(input_file, sep='\t', chunksize=chunksize) as reader:
        for chunk in reader:
            # Filter this chunk
            chunk_filtered = chunk[chunk[id_column].isin(de_values)]
            
            # Write out the filtered rows
            chunk_filtered.to_csv(
                output_file,
                sep='\t',
                index=False,
                header=first_chunk,  # write header only for the first chunk
                mode='a'             # append to the file
            )
            first_chunk = False

old_file = '../../../../zbmath_data_dump20231221-160126_hash.csv'
new_file = '../../../../zbmath_data_dump20240912-142641_hash.csv'
new_data_file = '../../../../zbmath_data_dump20240912-142641.csv'
old_name = old_file.split('/')[-1].split('-')[0]
new_name = new_file.split('/')[-1].split('-')[0]
old_de = read_de_numbers_csv(old_file)
new_de = read_de_numbers_csv(new_file)

old_ones_not_in_new = [int(x) for x in list(old_de - new_de)]
new_ones_not_in_old = [int(x) for x in list(new_de - old_de)]
with open(f'deleted_{old_name}_TO_{new_name}.tsv', 'a+') as f:
    f.write("de_number\n")
    for de in old_ones_not_in_new:
        f.write(str(de)+'\n')
subset_large_file_by_ids(
    input_file=new_data_file,
    output_file=f'new_{old_name}_TO_{new_name}.tsv',
    de_values=new_ones_not_in_old,
    id_column="de_number",
    chunksize=100000
)

/tmp/ipykernel_12357/3098516862.py:26: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in reader:
/tmp/ipykernel_12357/3098516862.py:26: DtypeWarning: Columns (16,17) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in reader:
/tmp/ipykernel_12357/3098516862.py:26: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in reader:
/tmp/ipykernel_12357/3098516862.py:26: DtypeWarning: Columns (7,8,9,12,13,14,15,16,17) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in reader:


In [9]:
len(old_ones_not_in_new)

2449

In [10]:
len(new_ones_not_in_old)

319375

In [11]:
type(list(new_ones_not_in_old)[1])

int